# Dataload - Frota Viva

### Importando as bibliotecas necessárias

In [1]:
from faker import Faker # para geração de dados ficticios
import random # para gerar numeros aleatorios
from datetime import datetime # para gerar os valores de data da tabela informações
import pandas as pd
from dotenv import load_dotenv
import sqlalchemy as sqla
import psycopg2
import os
import bcrypt as bcr
from passlib.hash import bcrypt
import re

# definindo lingua do faker para portugues 
# e a seed com o mesmo numero em ambas
# para a geração do mesmo padrão de valores
fake = Faker("pt-BR")
Faker.seed()
random.seed(1)

### Função para tratar os dados criados pelo Faker

In [2]:
def desmascarar(s):
    s = str(s)
    return re.sub("[\\(\\) \\.\\-\\+\\/]" , "", s) 


def criptografarSenha(senha):
    senha = str(senha)

    # codifica em bytes
    senha_bytes = senha.encode('utf-8')

    # garante que não ultrapassa 72 bytes (limite do bcrypt)
    if len(senha_bytes) > 72:
        senha_bytes = senha_bytes[:72]
        # evita cortar byte UTF-8 no meio
        while True:
            try:
                senha_bytes.decode('utf-8')
                break
            except UnicodeDecodeError:
                senha_bytes = senha_bytes[:-1]

    # gera o hash com ident "2a" e custo 10
    salt = bcr.gensalt(rounds=10, prefix=b"2a")
    hashed = bcr.hashpw(senha_bytes, salt)

    return hashed.decode('utf-8')


### Código para criar os insert que vai no documento scriptDataloadV2, a mostra os possíveis logins deste Dataload no CRUD

In [ ]:
loginsSalvos=pd.DataFrame(columns=['email','senha'])
qt_motorista=0
insertEmpresa = "\nINSERT INTO Empresa (tipo_empresa, cnpj, email, senha, nome, cep, rua, complemento, numero, pais, estado, cidade) VALUES\n"
insertMotorista = "\nINSERT INTO Motorista (nome, email, cpf, senha, id_empresa) VALUES\n"
insertTelefoneMotorista = "\nINSERT INTO Telefone_Motorista(telefone_motorista, id_motorista) VALUES\n"
insertFrota = "\nINSERT INTO Frota( tamanho_frota, tipo_frota, regiao, id_empresa ) VALUES\n"
insertCaminhao = "\nINSERT INTO Caminhao( placa, status, kms_rodados, modelo, capacidade, id_frota) VALUES\n"
insertCaminhaoMotorista = "\nINSERT INTO Caminhao_Motorista(id_caminhao, id_motorista) VALUES \n"
insertManutencao = "\nINSERT INTO Manutencao(dt_cadastro, dt_conclusao, tipo_manutencao, custo, ultimo_motorista, descricao_servico, id_caminhao) VALUES \n"
insertEntrega = "\nINSERT INTO Entrega( descricao_produto, dt_pedido, dt_entrega, cep, rua, complemento, numero, pais, estado, cidade, id_motorista) VALUES\n"
insertInformacoes = "\nINSERT INTO Informacoes(temp_ar_admissao, posicao_acelerador, tempo_motor_ligado, carga_motor, rpm_motor, dt_hora_leitura, pressao_pneu, sensores_oxigenio, qtd_combustivel, temp_arrefecimento, pressao_coletor_admissao, id_caminhao) VALUES\n"
insertTelefoneEmpresa = "\nINSERT INTO Telefone_Empresa( telefone_empresa, id_empresa) VALUES\n"

frotaAcumulado=2
caminhaoAcumulado=3



for idEmpresa in range(1,6):
    logins=pd.DataFrame(columns=['email','senha'])
    logins.loc[0,'email']=fake.company_email()
    logins.loc[0,'senha']=fake.password()
    loginsSalvos = pd.concat([loginsSalvos,logins],ignore_index=True)

    insertEmpresa += f"('{random.choice(['Transportadora', 'Logistica', 'Distribuidora'])}', '{desmascarar(fake.cnpj())}', '{logins.loc[0,'email']}', '{criptografarSenha(logins.loc[0,'senha'])}', '{fake.company()}', '{desmascarar(fake.postcode())}', '{fake.street_name()}', 'Complemento não informado', {random.randint(1, 9999)}, 'Brasil', 'SP', '{fake.city()}'),\n"

    
    qt_motorista+=10
    
    reducao=qt_motorista-1

    for idMotorista in range(qt_motorista-9,qt_motorista+1):

        
        
        insertMotorista += f"('{fake.name()}', '{fake.email()}', '{desmascarar(fake.cpf())}', '{criptografarSenha(fake.password())}', {idEmpresa}),\n"
        for i in range(2):
            insertTelefoneMotorista+=f"('{fake.bothify(text='##9########')}' ,{idMotorista}),\n"


    for idFrota in range(frotaAcumulado-1,frotaAcumulado+1):
        frotaAcumulado+=1
        insertFrota += f"({3}, '{random.choice(['Comercial', 'Entregas', 'Empresarial', 'Executiva', 'Serviços Públicos'])}', 'SP', {idEmpresa}),\n"
        
        for idCaminhao in range(caminhaoAcumulado-2,caminhaoAcumulado+1):
            caminhaoAcumulado+=1

            insertCaminhao += f"('{fake.bothify(text='???#?##').upper()}', '{fake.random_element(elements=('I', 'A', 'M'))}', {random.randint(1,9999)}, '{fake.random_element(elements=('FURGOES','CARREGA-TUDO','TANQUE','LINHA-LEVE', 'GRANELEIRO', 'BASCULANTE', 'FLORESTAL', 'PORTA-CONTAINER'))}', {random.randint(1,999)}, {idFrota}),\n"

            for i in range(2):   
                insertCaminhaoMotorista += f"({idCaminhao}, {random.randint(qt_motorista-9,qt_motorista)} ),\n"

            for i in range(2):

                dt_cadastro =fake.date_between_dates(date_start='-2y')
                dt_conclusaoBruta=fake.date_between_dates(date_start=dt_cadastro).strftime("%Y-%m-%d")

                insertManutencao += f"('{dt_cadastro}', '{fake.random_element(elements=(dt_conclusaoBruta, 'NULL'))}', '{fake.random_element(elements=('Corretiva','Preventiva'))}', {random.randint(100,10000)}, {random.randint(qt_motorista-9,qt_motorista)}, '{fake.text(50)}', {idCaminhao}),\n"
                
            for i in range(3):
                dt_pedido=fake.date_this_year();
                dtEntregaBruta =fake.date_between(start_date=dt_pedido, end_date='today').strftime("%Y-%m-%d")

                insertEntrega += f"('{fake.sentence(4)}', '{dt_pedido}', '{fake.random_element(elements=(dtEntregaBruta, 'NULL'))}', '{desmascarar(fake.postcode())}', '{fake.street_name()}', '{fake.sentence(3)}', {random.randint(1,999)}, 'Brasil', 'SP', '{fake.city()}', {random.randint(qt_motorista-9,qt_motorista)} ),\n"

            
            for i in range(2):
                data = fake.date_object() # gerando data 
                hora = fake.time_object() # gerando hora
                datacombinada = datetime.combine(data, hora) # juntando os dois em uma string

                insertInformacoes += f"({random.randint(15, 45)}, {random.randint(0, 100)}, '{fake.time_object()}', {random.randint(0, 100)}, {random.randint(600, 3000)}, '{datacombinada.strftime('%Y-%m-%d %H:%M:%S')}', {round(random.uniform(28.0, 36.0), 2)}, {random.randint(0, 1)}, {random.randint(0, 200)}, {random.randint(70, 110)}, {round(random.uniform(0.8, 2.5), 2)}, {idCaminhao}),\n"
		        
    for i in range(2):
        insertTelefoneEmpresa += f"('{fake.bothify(text='##9########')}' ,{idEmpresa}),\n"

with open("../sql/scriptDataloadV2.sql", "w", encoding='utf-8') as o:
    o.write("-- Script Dataload")  

for insert in [insertEmpresa,insertMotorista,insertTelefoneMotorista,insertFrota,insertCaminhao,insertCaminhaoMotorista,insertManutencao,insertEntrega,insertInformacoes,insertTelefoneEmpresa]:
    insert = insert[:len(insert)-2]
    insert+=";"
    insertCorrigido = insert.replace("'NULL'","NULL")
    try:
        with open("../sql/scriptDataloadV2.sql", "a", encoding='utf-8') as o:
            o.write(insertCorrigido)
    except FileNotFoundError:
        print('Arquivo não encontado ao escrever o insert da tabela motorista')

display(loginsSalvos)

,email,senha
0,ryan73@da.br,%K_LbfVAx2
1,antonyazevedo@leao.com,w@8SePNiaE
2,pastorana-cecilia@sales.com,JWD#95Kv%1
3,lizsouza@porto.br,*I4ALiGb*9
4,cmoraes@almeida.com,9^4*weEbY8
